In [1]:
import numpy as np
import matplotlib.pyplot as plt

import sys
sys.path.append("..")
sys.path.append("../src")

import rna_transform.utils as utils
from rna_transform.rna_helper import RNAHelper
from qspright.utils import best_convex_underestimator
from rna_transform.rna_tests import run_accuracy_tests
import itertools

In [2]:
np.random.seed(20)

positions = np.sort(np.random.choice(50, size = 20, replace=False))
print("positions: ", positions)

query_args = {
    "query_method": "complex",
    "delays_method": "nso",
    "num_subsample": 3,
    "num_random_delays": 3,
    "b": 8
}

helper = RNAHelper(positions, subsampling = True, query_args = query_args)
n = helper.n
q = helper.q

print("n = {}, N = {:.2e}".format(n, q ** n))

positions:  [ 0  2 12 13 14 16 17 18 19 23 25 26 30 34 36 37 40 41 43 45]
{'signal': 'results/rna_subsampled.pickle'}
{'n': 20, 'q': 4, 'positions': array([ 0,  2, 12, 13, 14, 16, 17, 18, 19, 23, 25, 26, 30, 34, 36, 37, 40,
       41, 43, 45]), 'query_args': {'query_method': 'complex', 'delays_method': 'nso', 'num_subsample': 3, 'num_random_delays': 3, 'b': 8}}
M Generation:0.00010228157043457031
D Generation:0.000125885009765625
Geneartion of query:  55.80017590522766  sec


100%|██████████| 4128768/4128768 [31:38<00:00, 2174.55it/s]


Geneartion of query:  55.187196254730225  sec


100%|██████████| 4128768/4128768 [30:23<00:00, 2263.62it/s]


KeyboardInterrupt: 

## Test QSPRIGHT with different parameters

Construct a grid of parameters. For each entry, run multiple test rounds. Compute the average for each parameter selection. 

In [ ]:
# iters = 1
# num_subsample_list = [4, 5, 6, 7]
# num_random_delays_list = [4, 6, 8, 10]
# b_list = [4, 5, 6, 7]

iters = 4
num_subsample_list = [4]
num_random_delays_list = [4]
b_list = [6]

result = run_accuracy_tests(helper, iters, num_subsample_list, num_random_delays_list, b_list)
    
sample_ratios, unique_sample_ratios, accs, hamming_ws = result

In [ ]:
for i, b in enumerate(b_list):
    print(b, np.mean(hamming_ws[:, :, i, :]))

In [ ]:
all_points = np.zeros(shape=[0,2])

for i, b in enumerate(b_list):
    s_values = np.mean(unique_sample_ratios[:, :, i, :], axis=-1).flatten()
    mse_values =  np.mean(accs[:, :, i, :], axis=-1).flatten()
    s_std = np.std(unique_sample_ratios[:, :, i, :], axis=-1).flatten()
    mse_std =  np.std(accs[:, :, i, :], axis=-1).flatten()
    
    plt.errorbar(s_values, mse_values, xerr= s_std, yerr= mse_std, label = "b = {}".format(b), fmt="o")
        
    all_points = np.concatenate((all_points, np.array([s_values, mse_values]).T), axis = 0)

if len(all_points) > 3:
    bcue = best_convex_underestimator(all_points)
    plt.plot(bcue[:,0], bcue[:,1], 'r--', lw=1.5, label="Best Cvx Underest.")

plt.xlabel('Unique Sample Ratio')
plt.ylabel('Prediction NMSE')
plt.legend()
plt.grid()
plt.savefig('figs/acc-vs-unique-sample-ratio.png')
plt.show()

In [ ]:
all_points = np.zeros(shape=[0,2])

for i, b in enumerate(b_list):
    s_values = np.mean(sample_ratios[:, :, i, :], axis=-1).flatten()
    mse_values =  np.mean(accs[:, :, i, :], axis=-1).flatten()
    s_std = np.std(sample_ratios[:, :, i, :], axis=-1).flatten()
    mse_std =  np.std(accs[:, :, i, :], axis=-1).flatten()

    plt.errorbar(s_values, mse_values, xerr= s_std, yerr= mse_std, label = "b = {}".format(b), fmt="o")

    all_points = np.concatenate((all_points, np.array([s_values, mse_values]).T), axis = 0)

if len(all_points) > 3:
    bcue = best_convex_underestimator(all_points)
    plt.plot(bcue[:,0], bcue[:,1], 'r--', lw=1.5, label="Best Cvx Underest.")

plt.xlabel('Total Sample Ratio')
plt.ylabel('Prediction NMSE')
plt.legend()
plt.grid()
plt.savefig('figs/acc-vs-total-sample-ratio.png')
plt.show()